The objective of this jupyter notebook is to count the distinct structure for 2 Al atoms in one cell with 36 T sites.

The codes have 14 sections.  
1. Modification of ATAT  
2. Prepare lattice input file (lat.in)  
3. Read lattice input file (lat.in)  
4. Functions to convert between fractional coordinates and xyz coordinates  
5. Prepare structure requirements input file (str_req.txt)  
6. Read structure requirements input file (str_req.txt)  
7. Function to find the site index for any given fractional coordinates  
8. Create str.out file  
9. Run corrdump to generate cluster list into cluster_list.csv file  
10. Read cluster_list.csv  
11. Add pair sites for each site in structure sites dataframe  
12. Randomly generate structure for any given Si/Al ratio  
13. Count clusters during copper titration  
14. Cu/Al for different Si/Al ratios  

#### 1. Modification of ATAT

Before we run the codes, we have to modify ATAT files to output the cluster list when we run corrdump to count clusters. Corrdump has many parameters. Here we will only set the maximum distance between two points within a pair (-2=[real]), with other parameters being default. The modification of ATAT contains 2 parts: one is in calccorr.c++ file and the other is in corrdump.c++ file. After the modifications, **corrdump will output the xyz coordinates for all the clusters in a structure inspite of the atom occupations.**

Note: If you also want to set other parameters, you may need to make more modifications.

a) In file calccorr.c++, please modify the following function (around line 200) as below. The modified lines have comments at the end.

b) In file corrdump.c++, please modify the following code block (around line 460). The modified lines have comments at the end.

c) After you modify the ATAT codes, you have to reinstall the ATAT **in terminal**. It should contain the following two commands.

Python codes start here

In [1]:
import pandas as pd
import numpy as np
import copy
from math import *
import os
import random
from collections import defaultdict
import csv
from functools import reduce
from copy import deepcopy
import pickle
from timeit import default_timer as timer
from matplotlib import pyplot as plt
%matplotlib inline

#### 2. Prepare lattice input file (lat.in)

The input file of lat.in contains the lattice parameters and all the possible types of atoms at each site. The detailed format of lat.in can be found in the ATAT manual (https://www.brown.edu/Departments/Engineering/Labs/avdw/atat/manual.pdf) pg. 36.  
 
The following information is from ATAT manual.  
Lattice file format:  
First, the coordinate system a,b,c is specified, either as  
[a] [b] [c] [alpha] [beta] [gamma]  

Then the lattice vectors u,v,w are listed, expressed in the coordinate system   just defined:  
[ua] [ub] [uc]  
[va] [vb] [vc]  
[wa] [wb] [wc]  

Finally, atom positions and types are given, expressed in the same coordinate system   as the lattice vectors:  
[atom1a] [atom1b] [atom1c] [atom1type]  
[atom2a] [atom2b] [atom2c] [atom2type]  
etc.  

In the lattice file:  
-The atom type is a comma-separated list of the atomic  
symbols of the atoms that can sit the lattice site.  
-In a binary, the first symbol listed is assigned a spin of -1.  
In general, ordering of the atom symbol corresponds to value of s=0,1,... in the table  
’Convention used to calculate the correlations’ below.  
-When only one symbol is listed, this site is ignored for the purpose  
of calculating correlations, but not for determining symmetry.  
-The atomic symbol ’Vac’ or ’Va’ is used to indicate a vacancy.  

#### 3. Read lattice input file (lat.in)
Read lat.in file and create a lattice dictionary which contains the lattice parameters, lattice vectors and fractional coordinates of all the sites in the lattics.

In [2]:
filepath='lat.in'
file = open(filepath, 'r')
lat = file.readlines()

lattice = {}
lattice['a'], lattice['b'], lattice['c'], lattice['alpha'], lattice['beta'], lattice['gamma'] = [float(number) for number in lat[0].split()]
lattice['u'] = [int(number) for number in lat[1].split()]
lattice['v'] = [int(number) for number in lat[2].split()]
lattice['w'] = [int(number) for number in lat[3].split()]

lattice_sites = pd.DataFrame(columns=['a', 'b', 'c', 'atom'])
for line in lat[4:]:
    site = line.split()[:3]
    atom_type = ''
    for atom in line.split()[3:]:
        atom_type += str(atom)
    site.append(atom_type)
    lattice_sites = lattice_sites.append(pd.DataFrame([site], columns=['a', 'b', 'c', 'atom']))
lattice_sites = lattice_sites.apply(pd.to_numeric, errors = 'ignore')

In [3]:
lattice

{'a': 13.675,
 'alpha': 90.0,
 'b': 13.675,
 'beta': 90.0,
 'c': 14.767,
 'gamma': 120.0,
 'u': [1, 0, 0],
 'v': [0, 1, 0],
 'w': [0, 0, 1]}

In [4]:
lattice_sites.head()

,a,b,c,atom
0,0.666967,0.106933,0.228233,"Si,Al"
0,0.893067,0.560033,0.228233,"Si,Al"
0,0.666967,0.560033,0.228233,"Si,Al"
0,0.439967,0.333033,0.228233,"Si,Al"
0,0.568667,0.137333,0.456033,O


#### 4. Functions to convert between fractional coordinates and xyz coordinates

In [5]:
def lattice_axes_xyz(lattic):
    '''
    function to prepare the xyz axes (a matrix)
                [ax] [ay] [az]
    axes_xyz =  [bx] [by] [bz]
                [cx] [cy] [cz]
    '''
    a = np.array([1,0,0]) * lattice['a']
    b = np.array([cos(lattice['gamma']/180*pi), sin(lattice['gamma']/180*pi), 0]) * lattice['b']
    c = np.cross(a, b)/np.linalg.norm(np.cross(a, b)) * lattice['c']
    return np.array((a,b,c))

In [6]:
def lattice_axes_abc(lattice):
    '''
    function to prepare the abc axes (a matrix)
                [ua] [ub] [uc]
    axes_abc =  [va] [vb] [vc]
                [wa] [wb] [wc]
    '''
    return np.array((lattice['u'], lattice['v'], lattice['w']))

In [7]:
def frac_to_xyz(axes_xyz, frac_coor):
    '''
    function to convert fractional coordinates to xyz coordinates
    '''
    return np.dot(frac_coor, axes_xyz)

In [8]:
def xyz_to_frac(axes_xyz, xyz_coor):
    '''
    function to convert xyz coordinates to fractional coordinates
    '''
    return np.dot(xyz_coor, np.linalg.inv(axes_xyz))

In [9]:
axes_abc = lattice_axes_abc(lattice)

In [10]:
axes_abc

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [11]:
axes_xyz = lattice_axes_xyz(lattice)

In [12]:
axes_xyz

array([[ 13.675    ,   0.       ,   0.       ],
       [ -6.8375   ,  11.8428974,   0.       ],
       [  0.       ,  -0.       ,  14.767    ]])

In [13]:
frac_to_xyz(axes_xyz, np.array((0.5, 0.5, 0)))

array([ 3.41875  ,  5.9214487,  0.       ])

In [14]:
xyz_to_frac(axes_xyz, np.array((6.8375, 0, 0)))

array([ 0.5,  0. ,  0. ])

#### 5. Prepare structure requirements input file (str_req.txt)
Before creating str_req.txt, we have to know all the cluster types within the maximum distance that we specified. We can run corrdump with only the lat.in file. A clusters.out file will be generated. It contains the information of all the symmetrically distinct clusters for the lattice that you specified in lat.in file. Based on the cluster order in the clusters.out file and the number of atoms(sites) containing in the cluster, we will define the cluster type as: 

cluster type = [nsite]-[order of the specific cluster in all nsite-clusters in clusters.out].

For example, for the first 2-body cluster in clusters.out, its cluster type is 2-1.  
For the second 2-body cluster in clusters.out, its cluster type is 2-2.

Then we will create a file which describe the structure requirements. It will include 3 parts: 1) the structure dimensions, 2) the clusters that cannot exist based on the rules we defined (*bad clusters*), and 3) the clusters of interest (*good clusters*, we want to count the number of them).  

The specific str_req.txt format:

First the structure vectors u,v,w are listed, expressed in the coordinate system just defined:  
[ua] [ub] [uc]  
[va] [vb] [vc]  
[wa] [wb] [wc]  

Next, the cluster types that cannot exist:  
[badtype1] [badtype2] [badtype3] [badtype4] etc.  

Finally, the cluster types of interest:  
[goodtype1] [goodtype2] [goodtype3] [goodtype4] etc.  

Here is an example of str_req.txt file.

2 0 0  
0 2 0  
0 0 2   
2-1, 2-2, 2-3, 2-4  
2-5, 2-6, 2-7, 2-8, 2-9, 2-10, 2-11  

#### 6. Read structure requirements input file (str_req.txt)
Generate structure dictionary containing structure dimensions, site coordinates, bad clusters (cannot exist) and good clusters (we want to count).

In [15]:
structure = {}

In [16]:
file = open('str_req.txt', 'r')
str_req = file.readlines()

structure_u = [int(number) for number in str_req[0].split()]
structure_v = [int(number) for number in str_req[1].split()]
structure_w = [int(number) for number in str_req[2].split()]

#bad_types = [s.replace(' ', '').replace('\n', '') for s in str_req[3].split(',')]
#good_types = [s.replace(' ', '').replace('\n', '') for s in str_req[4].split(',')]

In [17]:
structure_u

[1, 0, 0]

In [18]:
nu = int(np.mean(structure_u)/np.mean(lattice['u']))
nv = int(np.mean(structure_v)/np.mean(lattice['v']))
nw = int(np.mean(structure_w)/np.mean(lattice['w']))
n_cell = nu * nv * nw

In [19]:
nu

1

In [20]:
O_unit_sites = lattice_sites[lattice_sites['atom']=='O']
O_sites = pd.DataFrame(columns=lattice_sites.columns)
for i in range(0, nu):
    for j in range(0, nv):
        for k in range(0, nw):
            sites = pd.DataFrame(columns=O_unit_sites.columns)
            delta = list(np.array(lattice['u'])*i + np.array(lattice['v'])*j + np.array(lattice['w'])*k)
            sites['a'] = O_unit_sites.a + delta[0]
            sites['b'] = O_unit_sites.b + delta[1]
            sites['c'] = O_unit_sites.c + delta[2]
            sites['atom'] = O_unit_sites.atom
            O_sites = O_sites.append(sites)

In [21]:
important_unit_sites =  lattice_sites[lattice_sites['atom']!='O']
structure_sites = pd.DataFrame(columns=lattice_sites.columns)
for i in range(0, nu):
    for j in range(0, nv):
        for k in range(0, nw):
            sites = pd.DataFrame(columns=important_unit_sites.columns)
            delta = list(np.array(lattice['u'])*i + np.array(lattice['v'])*j + np.array(lattice['w'])*k)
            sites['a'] = important_unit_sites.a + delta[0]
            sites['b'] = important_unit_sites.b + delta[1]
            sites['c'] = important_unit_sites.c + delta[2]
            sites['atom'] = important_unit_sites.atom
            structure_sites = structure_sites.append(sites)

In [22]:
structure_sites.reset_index(drop=True, inplace=True)
structure_sites['site_index'] = structure_sites.index

In [23]:
structure_sites.head(5)

,a,b,c,atom,site_index
0,0.666967,0.106933,0.228233,"Si,Al",0
1,0.893067,0.560033,0.228233,"Si,Al",1
2,0.666967,0.560033,0.228233,"Si,Al",2
3,0.439967,0.333033,0.228233,"Si,Al",3
4,0.666367,0.106633,0.438433,"Si,Al",4


In [24]:
structure['u'] = structure_u
structure['v'] = structure_v
structure['w'] = structure_w
structure['nu'] = nu
structure['nv'] = nv
structure['nw'] = nw
structure['sites'] = structure_sites

#### 7. Function to find the site index for any given fractional coordinates

In [25]:
def find_site_index(axes_abc, structure, frac_coor):
    df = structure['sites']
    nu, nv, nw = structure['nu'],structure['nv'],structure['nw']
    
    fu, fv, fw = np.dot(frac_coor, np.linalg.inv(axes_abc))
    
    #translate the site into the structure by subtracting multiple structure vector on each dimension
    fu -= (fu//nu)*nu
    fv -= (fv//nv)*nv
    fw -= (fw//nw)*nw
    
    fa, fb, fc = np.dot(np.array((fu, fv, fw)), axes_abc)
    
    site_index = df[(abs(df.a-fa) < 0.001) & (abs(df.b-fb) < 0.001) & (abs(df.c-fc) < 0.001)].index
    if len(site_index) == 0:
        print("Error! Cannot find the site index in the structure.")
        return np.nan
    elif len(site_index) < 1:
        print("Error! Find multiple site indices in the structure.")
        return np.nan
    else:
        return site_index[0]

In [26]:
find_site_index(axes_abc, structure, [ -3.00016543e-04,   7.73299784e-01,   1.10509988e+00])

7

In [27]:
frac_coor = xyz_to_frac(axes_xyz, [15.22711,2.68122,1.55200])

In [28]:
find_site_index(axes_abc, structure, frac_coor)

33

#### 8. Create str.out file  
A str.out file is created based on the structure dimensions with Si on all sites. Actually the coordinates of all the clusters(cluster list) won't change in regards to the atom type on each site. Here we just put Si on all sites. 

In [29]:
if os.path.isfile('str.out'):
    os.remove('str.out')
with open('str.out', 'a') as str_file:
    str_file.write('{} {} {} {} {} {}\n'.format(lattice['a'], lattice['b'], lattice['c'], int(lattice['alpha']), int(lattice['beta']), int(lattice['gamma'])))
    str_file.write('{} {} {}\n'.format(int(structure['u'][0]),int(structure['u'][1]),int(structure['u'][2])))
    str_file.write('{} {} {}\n'.format(int(structure['v'][0]),int(structure['v'][1]),int(structure['v'][2])))
    str_file.write('{} {} {}\n'.format(int(structure['w'][0]),int(structure['w'][1]),int(structure['w'][2])))
    for index, row in structure['sites'].iterrows():
        str_file.write('{} {} {} Si\n'.format(row.a, row.b, row.c))

#### 9. Run corrdump to generate cluster list into cluster_list.csv file  
The cluster list contains the coordinates of all the clusters.

In [30]:
# Here we define the maximum distance between 2 atoms in a 2-body cluster is 6.2.
if os.path.isfile('cluster_list.csv'):
    os.remove('cluster_list.csv')
!corrdump -2 = 11 >> cluster_list.csv

#### 10. Read cluster_list.csv

The format for the file cluster_list.csv is as below.  
For each cluster type:  
First, the number of site in the cluster is given:  
[nsite]  
  
Next, the multiplicity (number of the cluster in the unit cell) is given:    
[multiplicity]  

Then, site xyz coordinates are listed for each cluster of that type:  
[cluster1site1x] [cluster1site1y] [cluster1site1z]  
[cluster1site2x] [cluster1site2y] [cluster1site2z]   
...  
[cluster1siteNx] [cluster1siteNy] [cluster1siteNz]  
[cluster2site1x] [cluster2site1y] [cluster2site1z]  
[cluster2site2x] [cluster2site2y] [cluster2site2z]   
...  
[cluster2siteNx] [cluster2siteNy] [cluster2siteNz]  
...  
...  
etc.  

Finally, the correlation function is given:  
[correlation function]  

Since all sites have Si in str.out, the correlation function is usually 0 or 1(for the first cluster type).

In [31]:
ncell = structure['nu']*structure['nv']*structure['nw']

In [32]:
clulist = []

with open('cluster_list.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=' ')
    for row in readCSV:
        if(len(row) == 0):
            continue
        elif(len(row) == 1):
            row[0] = row[0].replace('\t', '')
            clulist.append(int(float(row[0])))
        else:
            temp = []
            for element in row:
                temp.append(float(element))
            clulist.append(temp)

Create a dictionary (called clusters) which contains 1) cluster types, 2) cluster type number for different number of sites, and 3) the site indices for all the clusters for each cluster type.  

clusters['types'] will list all the cluster types.  
clusters['nsite_type_numbers'] will list the number of the types for nsite.  
clusters['2-1'] will list all the clusters (consist of site indices) of type 2-1.  

In [ ]:
clusters = {}
nsite_type_numbers = {}
cluster_types = []

i = 0
nsite = 0
cluster_index = 1
while(i < len(clulist)):
    #read n_site
    new_nsite = clulist[i]
    i += 1
    
    #pass first three lines where the cluster contains 0 site
    if (new_nsite == 0):
        i+=2
        continue
    
    #if the cluster has more sites than the previous cluster, reset nsite and cluster index
    if (new_nsite != nsite):
        nsite = new_nsite
        cluster_index = 1
        
    #set the cluster type = [nsite]-[order of the specific cluster in all nsite-clusters].
    cluster_type = str(nsite)+'-'+str(cluster_index)
    cluster_types.append(cluster_type)
    
    #initialize a empty set called temp 
    temp = set()
        
    #read multiplicity
    multiplicity = clulist[i]
    i += 1
    
    #go through all the clusters
    n_cluster = multiplicity *ncell
    for j in range(n_cluster):
        cluster = set()
        for k in range(nsite):
            point = []
            for element in clulist[i]:
                point.append(float(element))
            frac_coor = xyz_to_frac(axes_xyz, point)
            site = find_site_index(axes_abc, structure, frac_coor)
            #print(point)
            #print(site)

            cluster.update([site])
            i += 1
        if frozenset(cluster) in temp:
            #print('Error! Repeated cluster found in line {}. The cluster is {}. The cluster type is {}'.format(i-1, cluster, cluster_type))
        temp.add(frozenset(cluster))
    
    #pass the line with correlation
    i += 1 
    
    #put the set of clusters in dictionary of clusters
    clusters[cluster_type] = temp
    nsite_type_numbers[str(nsite)] = cluster_index
    
    #count cluster types
    cluster_index += 1   

clusters['cluster_types'] = cluster_types
clusters['nsite_type_numbers'] = nsite_type_numbers

In [37]:
clusters['nsite_type_numbers']

{'1': 1, '2': 51}

In [39]:
clusters['2-1']

{frozenset({8, 14}),
 frozenset({18, 25}),
 frozenset({0, 13}),
 frozenset({4, 22}),
 frozenset({26, 29}),
 frozenset({31, 34}),
 frozenset({10, 20}),
 frozenset({1, 2}),
 frozenset({24, 27}),
 frozenset({30, 33}),
 frozenset({6, 12}),
 frozenset({11, 17}),
 frozenset({5, 23}),
 frozenset({7, 21}),
 frozenset({9, 15}),
 frozenset({32, 35}),
 frozenset({3, 28}),
 frozenset({16, 19})}

In [85]:
temp = set()
pre_clu = set()
for i in range(51):
    #print(i)
    pair = random.sample(clusters['2-{}'.format(i+1)], 1)[0]
    #print(pair)
    if not(pair in pre_clu):
        temp.add(pair)
        
    pre_clu.update(clusters['2-{}'.format(i+1)])

In [86]:
len(temp)

25

In [87]:
temp

{frozenset({2, 16}),
 frozenset({26, 30}),
 frozenset({12, 25}),
 frozenset({24, 30}),
 frozenset({19, 23}),
 frozenset({20, 24}),
 frozenset({16, 34}),
 frozenset({7, 17}),
 frozenset({31, 34}),
 frozenset({17, 18}),
 frozenset({1, 27}),
 frozenset({5, 9}),
 frozenset({6, 26}),
 frozenset({23, 30}),
 frozenset({10, 29}),
 frozenset({6, 31}),
 frozenset({1, 18}),
 frozenset({19, 28}),
 frozenset({14, 30}),
 frozenset({24, 29}),
 frozenset({4, 17}),
 frozenset({11, 19}),
 frozenset({28, 29}),
 frozenset({17, 29}),
 frozenset({13, 28})}